In [2]:
from datasets import load_dataset, DatasetDict, Dataset

In [3]:
from transformers import (
    AutoTokenizer,
    AutoConfig,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer
)
import torch.nn as nn

from peft import PeftModel, PeftConfig, get_peft_model, LoraConfig

In [4]:
import logging
logger = logging.getLogger(__name__)
logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.INFO, datefmt='%I:%M:%S')

In [5]:
import torch
import numpy as np
import evaluate

In [6]:
modelName = 'google-bert/bert-base-uncased'

In [7]:
id2label = {0: "Safe", 1: "Not Safe"}
label2id = {"Safe": 0, "Not Safe": 1}

model = AutoModelForSequenceClassification.from_pretrained(
    modelName, num_labels=2, id2label=id2label, label2id=label2id
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [9]:
dataset = load_dataset("csv", data_files="././phishing_dataset_full.csv")
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 100000
    })
})

In [10]:
dataset = dataset['train'].train_test_split(test_size=0.2)
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 80000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 20000
    })
})

In [11]:
dataset['test']["text"]

['https://www.cnn.com',
 'https://www.microsoft.com',
 'https://www.zoom.us',
 'https://www.wikipedia.org',
 'https://www.paypal.com',
 'https://www.google.com',
 'https://www.etsy.com',
 'https://www.reddit.com',
 'https://www.google.com',
 'https://www.ebay.com',
 'https://www.airbnb.com',
 'https://www.wikipedia.org',
 'https://www.linkedin.com',
 'https://www.youtube.com',
 'http://backupsync.biz/profile',
 'http://datacrypto.biz/update',
 'http://paypalfinance.edu/login',
 'http://twitterstore.net/password',
 'http://communityupdate.biz/profile',
 'https://www.stackoverflow.com',
 'https://www.youtube.com',
 'https://www.stackoverflow.com',
 'https://www.zoom.us',
 'https://www.facebook.com',
 'http://paypalcrypto.gov/account',
 'https://www.amazon.com',
 'https://www.instagram.com',
 'http://instagramforum.com/account',
 'https://www.linkedin.com',
 'http://infocommunity.edu/secure',
 'http://memberhelp.org/confirm',
 'http://shopbuy.info/confirm',
 'https://www.microsoft.com',
 

In [12]:
tokenizer = AutoTokenizer.from_pretrained(modelName)

In [13]:
# Freeze parameters:
for name, param in model.base_model.named_parameters():
    logger.info(name)
    param.requires_grad = False


06:02:49 INFO:embeddings.word_embeddings.weight
06:02:49 INFO:embeddings.position_embeddings.weight
06:02:49 INFO:embeddings.token_type_embeddings.weight
06:02:49 INFO:embeddings.LayerNorm.weight
06:02:49 INFO:embeddings.LayerNorm.bias
06:02:49 INFO:encoder.layer.0.attention.self.query.weight
06:02:49 INFO:encoder.layer.0.attention.self.query.bias
06:02:49 INFO:encoder.layer.0.attention.self.key.weight
06:02:49 INFO:encoder.layer.0.attention.self.key.bias
06:02:49 INFO:encoder.layer.0.attention.self.value.weight
06:02:49 INFO:encoder.layer.0.attention.self.value.bias
06:02:49 INFO:encoder.layer.0.attention.output.dense.weight
06:02:49 INFO:encoder.layer.0.attention.output.dense.bias
06:02:49 INFO:encoder.layer.0.attention.output.LayerNorm.weight
06:02:49 INFO:encoder.layer.0.attention.output.LayerNorm.bias
06:02:49 INFO:encoder.layer.0.intermediate.dense.weight
06:02:49 INFO:encoder.layer.0.intermediate.dense.bias
06:02:49 INFO:encoder.layer.0.output.dense.weight
06:02:49 INFO:encoder.

In [14]:
# Unfreeze some parameters:
for name, param in model.base_model.named_parameters():
    if "pooler" in name:
        logger.info(name)
        param.requires_grad = True

06:02:49 INFO:pooler.dense.weight
06:02:49 INFO:pooler.dense.bias


In [15]:
model.classifier.weight

Parameter containing:
tensor([[ 0.0066,  0.0061,  0.0325,  ...,  0.0164,  0.0039, -0.0274],
        [ 0.0250,  0.0081,  0.0074,  ..., -0.0200,  0.0098,  0.0007]],
       requires_grad=True)

In [16]:
model.base_model.pooler.dense.weight

Parameter containing:
tensor([[-0.0013, -0.0381, -0.0158,  ...,  0.0244, -0.0008,  0.0240],
        [ 0.0020,  0.0151,  0.0033,  ...,  0.0180, -0.0023,  0.0231],
        [-0.0386,  0.0145,  0.0621,  ...,  0.0374, -0.0105, -0.0395],
        ...,
        [-0.0111,  0.0136,  0.0541,  ...,  0.0666,  0.0017, -0.0090],
        [ 0.0001,  0.0024, -0.0125,  ...,  0.0046, -0.0014, -0.0079],
        [ 0.0415,  0.0751,  0.0305,  ...,  0.0317,  0.0479,  0.0080]],
       requires_grad=True)

In [17]:
model.base_model.embeddings.word_embeddings.weight.requires_grad

False

In [18]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

tokenized_dataset = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/80000 [00:00<?, ? examples/s]

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

In [19]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 80000
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 20000
    })
})

In [20]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [21]:
accuracy = evaluate.load("accuracy")
auc_score = evaluate.load("roc_auc")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred

    probabilities = np.exp(predictions) / np.exp(predictions).sum(-1, keepdims=True)
    positive_class_probs = probabilities[:, 1]
#     logging.info(positive_class_probs)
    auc = np.round(auc_score.compute(prediction_scores=positive_class_probs,
                                    references=labels)['roc_auc'], 3)

    predicted_classes = np.argmax(predictions, axis=1)
#     logging.info(predicted_classes)

    acc = np.round(accuracy.compute(predictions=predicted_classes,
                                    references=labels)['accuracy'], 3)

    return {"Accuracy": acc, "AUC": auc}

In [22]:
lr=2e-4
batch_size=16
num_epochs=3

training_args=TrainingArguments(
    output_dir=modelName + "-RETRAIN",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    logging_strategy="epoch",
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,

    gradient_accumulation_steps=4,
    warmup_steps=2
)

In [23]:
import os
import wandb


os.environ["WANDB_MODE"] = "offline"
wandb.init(
    project="my-awesome-project")

In [24]:
trainer=Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

In [25]:
# trainer.train()

In [26]:
modelFT = AutoModelForSequenceClassification.from_pretrained(
    "./google-bert/bert-base-uncased-RETRAIN/checkpoint-6250", num_labels=2, id2label=id2label, label2id=label2id
)
modelFT.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [27]:
modelFT.base_model.pooler.dense.weight

Parameter containing:
tensor([[ 0.0446, -0.0400, -0.0211,  ...,  0.0245,  0.0138,  0.0311],
        [-0.0476,  0.0127, -0.0014,  ...,  0.0076, -0.0114,  0.0263],
        [ 0.0020,  0.0155,  0.0565,  ...,  0.0299,  0.0097, -0.0286],
        ...,
        [ 0.0229,  0.0195,  0.0531,  ...,  0.0672,  0.0130, -0.0083],
        [ 0.0489,  0.0083, -0.0118,  ...,  0.0055,  0.0099, -0.0017],
        [-0.0051,  0.0713,  0.0322,  ...,  0.0321,  0.0365,  0.0028]],
       requires_grad=True)

In [28]:
modelFT.classifier.weight

Parameter containing:
tensor([[ 0.0112, -0.0614,  0.0133,  ...,  0.0240,  0.0577, -0.0578],
        [-0.0369,  0.0230, -0.0444,  ..., -0.0502, -0.0186,  0.0218]],
       requires_grad=True)

In [29]:
tokenizerFT = AutoTokenizer.from_pretrained("./google-bert/bert-base-uncased-RETRAIN/checkpoint-6250", use_fast=True)

In [40]:
prompt = "http\\www.ya.ru"
inputs = tokenizerFT(prompt, return_tensors="pt")
outputs = modelFT(inputs["input_ids"])
out = nn.Softmax(dim=1)(outputs["logits"])
logger.info(out)

06:15:38 INFO:tensor([[0.8346, 0.1654]], grad_fn=<SoftmaxBackward0>)


In [41]:
for i, prompt in enumerate(dataset['test']["text"][:30]):
    inputs = tokenizerFT(prompt, return_tensors="pt")
    outputs = modelFT(inputs["input_ids"])
    out = nn.Softmax(dim=1)(outputs["logits"])
    outMax = torch.argmax(out)
    label = dataset['test']["label"][i]
    logger.info(f"Prob: {outMax}, Label: {label}")

07:33:17 INFO:Prob: 0, Label: 0
07:33:18 INFO:Prob: 0, Label: 0
07:33:18 INFO:Prob: 0, Label: 0
07:33:18 INFO:Prob: 0, Label: 0
07:33:18 INFO:Prob: 0, Label: 0
07:33:18 INFO:Prob: 0, Label: 0
07:33:18 INFO:Prob: 0, Label: 0
07:33:18 INFO:Prob: 0, Label: 0
07:33:18 INFO:Prob: 0, Label: 0
07:33:18 INFO:Prob: 0, Label: 0
07:33:19 INFO:Prob: 0, Label: 0
07:33:19 INFO:Prob: 0, Label: 0
07:33:19 INFO:Prob: 0, Label: 0
07:33:19 INFO:Prob: 0, Label: 0
07:33:19 INFO:Prob: 1, Label: 1
07:33:19 INFO:Prob: 1, Label: 1
07:33:19 INFO:Prob: 1, Label: 1
07:33:19 INFO:Prob: 1, Label: 1
07:33:19 INFO:Prob: 1, Label: 1
07:33:19 INFO:Prob: 0, Label: 0
07:33:19 INFO:Prob: 0, Label: 0
07:33:20 INFO:Prob: 0, Label: 0
07:33:20 INFO:Prob: 0, Label: 0
07:33:20 INFO:Prob: 0, Label: 0
07:33:20 INFO:Prob: 1, Label: 1
07:33:20 INFO:Prob: 0, Label: 0
07:33:20 INFO:Prob: 0, Label: 0
07:33:20 INFO:Prob: 1, Label: 1
07:33:20 INFO:Prob: 0, Label: 0
07:33:20 INFO:Prob: 1, Label: 1


In [36]:
model.to("cpu")

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [37]:
for prompt in dataset['test']["text"][:30]:
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model(inputs["input_ids"])
    out = nn.Softmax(dim=1)(outputs["logits"])
    logger.info(out)

06:04:42 INFO:tensor([[0.4265, 0.5735]], grad_fn=<SoftmaxBackward0>)
06:04:42 INFO:tensor([[0.4418, 0.5582]], grad_fn=<SoftmaxBackward0>)
06:04:42 INFO:tensor([[0.4484, 0.5516]], grad_fn=<SoftmaxBackward0>)
06:04:42 INFO:tensor([[0.4257, 0.5743]], grad_fn=<SoftmaxBackward0>)
06:04:42 INFO:tensor([[0.4362, 0.5638]], grad_fn=<SoftmaxBackward0>)
06:04:42 INFO:tensor([[0.4343, 0.5657]], grad_fn=<SoftmaxBackward0>)
06:04:42 INFO:tensor([[0.4485, 0.5515]], grad_fn=<SoftmaxBackward0>)
06:04:42 INFO:tensor([[0.4423, 0.5577]], grad_fn=<SoftmaxBackward0>)
06:04:42 INFO:tensor([[0.4343, 0.5657]], grad_fn=<SoftmaxBackward0>)
06:04:42 INFO:tensor([[0.4362, 0.5638]], grad_fn=<SoftmaxBackward0>)
06:04:42 INFO:tensor([[0.4371, 0.5629]], grad_fn=<SoftmaxBackward0>)
06:04:42 INFO:tensor([[0.4257, 0.5743]], grad_fn=<SoftmaxBackward0>)
06:04:42 INFO:tensor([[0.4217, 0.5783]], grad_fn=<SoftmaxBackward0>)
06:04:42 INFO:tensor([[0.4310, 0.5690]], grad_fn=<SoftmaxBackward0>)
06:04:42 INFO:tensor([[0.4312, 0.5